# Script para calcular diferencias entre dos informes

Este script será utilizado para calcular las diferencias entre dos informes de capital, es decir, entregará un dataframe con nombre, rut, credito, debito y saldo, donde los valores corresponderán a la resta del primer informe con el segundo.

NOTA: El informe que va primero debe ser el último en fecha para no obtener diferencias negativas.

In [1]:
import pandas as pd
def tonumberNeg(text):
    text = text.replace('.', '')
    text = text.replace(',', '')
    return -int(text)

def tonumberPos(text):
    text = text.replace('.', '')
    text = text.replace(',', '')
    return int(text)
def leerInforme(nombre):
    df = pd.read_csv("csv/{}.csv".format(nombre), delimiter=",")
    # Dejar solo nombre y saldo
    saldos = df[(df['Vencto.'] == 'Total') | (df['Vencto.'] == 'TOTAL')][['TP.1', 'Unnamed: 8', 'Créditos', 'Saldo']]
    # Dejar solo nombre y rut
    ruts = df[(~df['TP'].isna()) & (df['TP']!='VO')][['TP', 'Vencto.']]
    # Cambiar nombres de columnas a nombres buenos
    ruts.columns = ['Rut', 'Nombre']
    saldos.columns = ['Nombre', 'Debito','Credito', 'Saldo']
    # Especificar el tipo a la columna nombre
    ruts['Nombre'] = ruts['Nombre'].astype('string')
    saldos['Nombre'] = saldos['Nombre'].astype('string')
    # Transformar columna saldo a numerica
    saldos['Saldo'] = saldos['Saldo'].apply(tonumberNeg)
    saldos['Debito'] = saldos['Debito'].fillna('0').apply(tonumberPos)
    saldos['Credito'] = saldos['Credito'].fillna('0').apply(tonumberPos)
    # Sumar saldos de nombres repetidos y quitar nombres repetidos con diferentes ruts
    saldos = saldos.groupby('Nombre').sum().reset_index()
    ruts = ruts[~ruts.duplicated(subset="Nombre", keep='first')]
    print('len de saldos: {}, len de ruts: {} '.format(len(saldos), len(ruts)))
    # ordenar ruts
    ruts = ruts.sort_values(by=['Nombre'])
    #saldos = saldos.sort_values(by=['Nombre']) para probar pero el error era blumel
    # Join de tablas saldos y ruts para crear Capital.
    capital = saldos.reset_index(drop=True).join(ruts.reset_index(drop=True), rsuffix="_")
    print("Aseguarse que las filas fueron bien unidas si las unicas filas no iguales en la realidad corresponden:")
    print(capital[capital['Nombre']!=capital['Nombre_']])
    # Quitar fila de nombre extra
    capital = capital[['Nombre', 'Debito', 'Credito', 'Saldo', 'Rut']]

    #Quitar los que tengan saldo 0
    capital = capital[(capital['Saldo']>0) | (capital['Saldo']<0)]
    # Transformar a string
    capital['Nombre'] = capital['Nombre'].astype('string')
    # Transformar rut a string, quitar espacios vacios y transformar a minuscula.
    capital['Rut'] = capital['Rut'].astype('string').apply(str.strip).apply(str.lower)

    return capital

## Recordar
- quitar primera parte del archivo csv, dejar como primera fila la que dice Fecha, ...
- quitar ultima fila.
- Se debe agregar el nombre utilizado en el sistema a todos los ruts sin nombre, esto se hace buscando la siguiente cadena en el texto: "Total","","" 
- Se debe areglar los nombres de aquellos que no coincidan entre el nombre en la fila de rut y en la fila de total. Esto se puede hacer revisando el listado que la siguiente celda entrega.
- Se deben arreglar los nombres que no coincidan con el anterior, esto se puede ver con la celda de nulos en rut_desp, arreglandolos uno a uno, a menos que sea un retiro, en ese caso se ejecuta la celda siguiente.

In [4]:
ant = "Diciembre"
nuevo = "Enero"

mes_anterior = ant[:3]
mes_nuevo = nuevo[:3]

anteriorMalo = "2024{}malo".format(ant.lower())
nuevoMalo    = "2025{}malo".format(nuevo.lower())

antMalo_df = leerInforme(anteriorMalo)
nuevoMalo_df = leerInforme(nuevoMalo)

len de saldos: 916, len de ruts: 916 
Aseguarse que las filas fueron bien unidas si las unicas filas no iguales en la realidad corresponden:
                       Nombre  Debito  Credito  Saldo         Rut  \
313  GLADYS BARRIENTOS ARANGO       0    21031  21031   9427736-1   
347   IGNACIO TORO BARRIENTOS       0    21031  21031  17580807-8   

              Nombre_  
313  GLADYS BARRIENTO  
347      IGNACIO TORO  
len de saldos: 918, len de ruts: 918 
Aseguarse que las filas fueron bien unidas si las unicas filas no iguales en la realidad corresponden:
                       Nombre  Debito  Credito  Saldo         Rut  \
315  GLADYS BARRIENTOS ARANGO       0    21690  21690   9427736-1   
349   IGNACIO TORO BARRIENTOS       0    21690  21690  17580807-8   

              Nombre_  
315  GLADYS BARRIENTO  
349      IGNACIO TORO  


In [5]:
merged_df = pd.merge(antMalo_df, nuevoMalo_df, on='Nombre', how='outer', suffixes=('_antes', '_desp'))
merged_df['diff'] = merged_df['Saldo_desp'].fillna(0) - merged_df['Saldo_antes'].fillna(0)
merged_df['diff_debito'] = merged_df['Debito_desp'].fillna(0) - merged_df['Debito_antes'].fillna(0)
merged_df['diff_credito'] = merged_df['Credito_desp'].fillna(0) - merged_df['Credito_antes'].fillna(0)

In [6]:
merged_df[merged_df["Rut_desp"].isna()]

,Nombre,Debito_antes,Credito_antes,Saldo_antes,Rut_antes,Debito_desp,Credito_desp,Saldo_desp,Rut_desp,diff,diff_debito,diff_credito
83,Alicia Ruth Cayulef Ovando,0.0,10000.0,10000.0,8475029-8,NaN,NaN,NaN,NaN,-10000.0,0.0,-10000.0
474,Jorge Hernan Calvo Rojas,0.0,9000.0,9000.0,5785737-4,NaN,NaN,NaN,NaN,-9000.0,0.0,-9000.0
484,KATHERINNE LISETTE CONTRERAS GALLO,0.0,210564.0,210564.0,15340483-6,NaN,NaN,NaN,NaN,-210564.0,0.0,-210564.0
648,MICHAEL ALEXIS CRISTOBAL CRISTOBAL,1240820.0,1624720.0,383900.0,16940649-9,NaN,NaN,NaN,NaN,-383900.0,-1240820.0,-1624720.0
669,Marcia Lucia Hueitra Silva,0.0,40000.0,40000.0,15728786-9,NaN,NaN,NaN,NaN,-40000.0,0.0,-40000.0
694,Natalia Clorinda Iturra Catribil,0.0,20000.0,20000.0,18198136-9,NaN,NaN,NaN,NaN,-20000.0,0.0,-20000.0
699,ONATAN HENRY BRAVO LLANCAO,1.0,0.0,-1.0,19195801-2,NaN,NaN,NaN,NaN,1.0,-1.0,0.0
743,PIQUARDT ANNA,0.0,331109.0,331109.0,c21fx9261,NaN,NaN,NaN,NaN,-331109.0,0.0,-331109.0
788,RONJA FISCHER,0.0,1192.0,1192.0,cc8f388,NaN,NaN,NaN,NaN,-1192.0,0.0,-1192.0


In [7]:
# Reemplazar los rut desp con rut antes si rut desp no existe.
# Esto ocurre cuando hay retiros de capital
merged_df.loc[merged_df["Rut_desp"].isna(), "Rut_desp"] = merged_df.loc[merged_df["Rut_desp"].isna(), "Rut_antes"]

In [8]:
columns = ['Nombre', 'Rut_desp', 'diff_debito', 'diff_credito', 'diff']
merged_df[columns].to_excel("diferencias/Diferencias{}{}.xlsx".format(mes_nuevo, mes_anterior))
merged_df[columns].to_csv("diferencias/Diferencias{}{}.csv".format(mes_nuevo, mes_anterior), index=False)